In [1]:
import optuna
from optuna import Trial

from math import sqrt
from typing import Tuple, List

import numpy as np
import pandas as pd
from mordred import Calculator, descriptors
#import openbabel
from openbabel import pybel
from PyBioMed.PyMolecule.fingerprint import CalculatePubChemFingerprint,CalculateECFP2Fingerprint
from rdkit import Chem
from rdkit.Chem.rdchem import Atom

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import StratifiedKFold, KFold


from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as G_Loader 
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import BatchNorm


# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
# Pytorch and Pytorch Geometric
import torch

import torch.nn as nn
from torch.nn import Linear
import torch.optim as optim
import torch.nn.functional as F # activation function
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as V_Loader # dataset management
%run ./graph_feature.ipynb 
%run ./dataset_processing.ipynb 

%run ./my_performances.ipynb 


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#load data descriptor 
# reload the data fingerprint 
#=======================================================
k=10
final_clean_desc_train=[]
final_clean_desc_val=[]
for i in range(k):
    final_clean_desc_train.append(np.load('final_clean_desc_train'+ str(i)+'.npy'))
    final_clean_desc_val.append(np.load('final_clean_desc_val' +str(i)+'.npy'))

final_clean_desc_test = np.load('final_clean_desc_test.npy')

In [2]:
k=10
final_clean_fingerp_PubChem_train=[]
final_clean_fingerp_PubChem_val=[]
for i in range(k):
    final_clean_fingerp_PubChem_train.append(np.load('final_clean_fingerp_PubChem_train'+ str(i)+'.npy'))
    final_clean_fingerp_PubChem_val.append(np.load('final_clean_fingerp_PubChem_val' +str(i)+'.npy'))

final_clean_fingerp_PubChem_test = np.load('final_clean_fingerp_PubChem_test.npy')

In [3]:
# reload the data fingerprint 
#=======================================================
k=10
final_clean_fingerp_ECFP2_train=[]
final_clean_fingerp_ECFP2_val=[]
for i in range(k):
    final_clean_fingerp_ECFP2_train.append(np.load('final_clean_fingerp_ECFP2_train'+ str(i)+'.npy'))
    final_clean_fingerp_ECFP2_val.append(np.load('final_clean_fingerp_ECFP2_val' +str(i)+'.npy'))

final_clean_fingerp_ECFP2_test = np.load('final_clean_fingerp_ECFP2_test.npy')

In [4]:
# reload the data fingerprint 
#=======================================================
k=10
final_clean_fingerp_MACCS_train=[]
final_clean_fingerp_MACCS_val=[]
for i in range(k):
    final_clean_fingerp_MACCS_train.append(np.load('final_clean_fingerp_MACCS_train'+ str(i)+'.npy'))
    final_clean_fingerp_MACCS_val.append(np.load('final_clean_fingerp_MACCS_val' +str(i)+'.npy'))

final_clean_fingerp_MACCS_test = np.load('final_clean_fingerp_MACCS_test.npy')

In [5]:
train_idx = np.load('train_indices.npy')
val_idx = np.load('val_indices.npy')
test_idx = np.load('test_indices.npy')

In [6]:
# load the output label 
total_train_targets =[]
total_validation_targets =[]
total_test_targets=[]
for i in range(k):
    total_train_targets.append(np.load('total_train_targets'+ str(i)+'.npy'))
    total_validation_targets.append(np.load('total_validation_targets' +str(i)+'.npy'))

total_test_targets= np.load('total_test_targets.npy')

# 1 .variance threshold

In [7]:
# create data fusion 
# data preprocessing, reduce size , data selection
# finding the monoton columns 
from sklearn.feature_selection import VarianceThreshold,chi2

In [36]:
# choose just only one fold for fitting the variance threshold
total_desc               = np.concatenate((final_clean_desc_train[0],final_clean_desc_val[0]), axis= 0)
total_fingerp_ECFP2      = np.concatenate((final_clean_fingerp_ECFP2_train[0], final_clean_fingerp_ECFP2_val[0]), axis= 0)
total_fingerp_PubChem    = np.concatenate((final_clean_fingerp_PubChem_train[0], final_clean_fingerp_PubChem_val[0]), axis= 0)
total_fingerp_MACCS      = np.concatenate((final_clean_fingerp_MACCS_train[0],  final_clean_fingerp_MACCS_val[0]), axis= 0)

In [37]:
total_desc.shape

(1258, 1613)

In [38]:
total_fingerp_ECFP2.shape

(1258, 1024)

In [39]:
total_fingerp_PubChem.shape

(1258, 881)

In [40]:
total_fingerp_MACCS.shape

(1258, 166)

In [66]:
# fit the variance threshold model 
# we'll get bad feature from variance threshold and chi2
variance_desc =VarianceThreshold(threshold=(0.05))
variance_fingerp_ECFP2 =VarianceThreshold(threshold=(0.05))
variance_fingerp_PubChem =VarianceThreshold(threshold=(0.05))
variance_fingerp_MACCS =VarianceThreshold(threshold=(0.05))

# always prepare the data in dataframe format 
pd_desc = pd.DataFrame(total_desc)
pd_fingerp_ECFP2 = pd.DataFrame(total_fingerp_ECFP2 )
pd_fingerp_PubChem = pd.DataFrame(total_fingerp_PubChem )
pd_fingerp_MACCS= pd.DataFrame(total_fingerp_MACCS )

# we got the model here
result_desc = variance_desc.fit(pd_desc)
result_fingerp_ECFP2 = variance_fingerp_ECFP2.fit(pd_fingerp_ECFP2)
result_fingerp_PubChem = variance_fingerp_PubChem.fit(pd_fingerp_PubChem)
result_fingerp_MACCS = variance_fingerp_MACCS.fit(pd_fingerp_MACCS)

In [67]:
group_desc_train     = []
group_ECFP2_train    = []
group_PubChem_train  = []
group_MACCS_train    = []

for data in final_clean_desc_train:
    group_desc_train.append(result_desc.transform(data))
for data in final_clean_fingerp_ECFP2_train:
    group_ECFP2_train.append(result_fingerp_ECFP2.transform(data))
for data in final_clean_fingerp_PubChem_train:
    group_PubChem_train.append(result_fingerp_PubChem.transform(data))
for data in final_clean_fingerp_MACCS_train:
    group_MACCS_train.append(result_fingerp_MACCS.transform(data))

In [82]:
group_desc_val     = []
group_ECFP2_val    = []
group_PubChem_val  = []
group_MACCS_val    = []

for data in final_clean_desc_val:
    group_desc_val.append(result_desc.transform(data))
for data in final_clean_fingerp_ECFP2_val:
    group_ECFP2_val.append(result_fingerp_ECFP2.transform(data))
for data in final_clean_fingerp_PubChem_val:
    group_PubChem_val.append(result_fingerp_PubChem.transform(data))
for data in final_clean_fingerp_MACCS_val:
    group_MACCS_val.append(result_fingerp_MACCS.transform(data))

In [84]:
final_clean_desc_test.shape

(315, 1613)

In [86]:
final_clean_fingerp_ECFP2_test.shape

(1, 315, 1024)

In [87]:
final_clean_fingerp_PubChem_test.shape

(1, 315, 881)

In [88]:
final_clean_fingerp_MACCS_test.shape

(1, 315, 166)

In [89]:
group_desc_test     = result_desc.transform(final_clean_desc_test)
group_ECFP2_test    = result_fingerp_ECFP2.transform(final_clean_fingerp_ECFP2_test[0])
group_PubChem_test  = result_fingerp_PubChem.transform(final_clean_fingerp_PubChem_test[0])
group_MACCS_test    = result_fingerp_MACCS.transform(final_clean_fingerp_MACCS_test[0])

In [69]:
final_clean_desc_train[0].shape

(1132, 1613)

In [76]:
final_clean_fingerp_ECFP2_train[0].shape

(1132, 1024)

In [77]:
final_clean_fingerp_PubChem_train[0].shape

(1132, 881)

In [78]:
final_clean_fingerp_MACCS_train[0].shape

(1132, 166)

In [71]:
group_desc_train[0].shape

(1132, 43)

In [90]:
group_desc_test.shape

(315, 43)

In [73]:
group_ECFP2_train[0].shape

(1132, 110)

In [74]:
group_PubChem_train[0].shape

(1132, 346)

In [75]:
group_MACCS_train[0].shape

(1132, 124)

# 2 .Chi-squared Test

In [96]:
# the saved data already scaled by min max scaler
# combine all first before doing selection.
fusion_data_train=[] 
fusion_data_val=[]
#fusion_data_test=[]

for i in range(10):
    fusion_data_train.append(np.concatenate((group_desc_train[i], group_ECFP2_train[i],group_PubChem_train[i],
                                            group_MACCS_train[i]),axis =1))
    fusion_data_val.append(np.concatenate((group_desc_val[i], group_ECFP2_val[i],group_PubChem_val[i],
                                            group_MACCS_val[i]),axis =1))

fusion_data_test = np.concatenate((group_desc_test, group_ECFP2_test,group_PubChem_test,
                                            group_MACCS_test),axis =1)

In [95]:
fusion_data_train[0].shape

(1132, 623)

In [97]:
fusion_data_val[0].shape

(126, 623)

In [98]:
fusion_data_test.shape

(315, 623)

In [102]:
# start the chi squared test
total_fusion = np.concatenate((fusion_data_train[0], fusion_data_val[0]), axis= 0)
label_fusion = np.concatenate((total_train_targets[0],total_validation_targets[0]),axis=0)
f_score_fusion = chi2(total_fusion, label_fusion)

In [105]:
pValue_fusion = pd.Series(f_score_fusion[1])

In [106]:
# get the bad features first 
# choose the small pvalue 
def get_bad_features(pValue,thr):
    list_bad_feature=[]
    for i in range(len(pValue)):
        if pValue[i] > thr:
            list_bad_feature.append(i)
    return list_bad_feature

# set the pvalue threshold to eliminate bad features 
thr1 = 0.1

In [107]:
list_bad_feature= get_bad_features(pValue_fusion, thr1) 

In [108]:
final_clean_fusion_data_train =[]
final_clean_fusion_data_val =[]

for data_train, data_val in zip (fusion_data_train,fusion_data_val):
    final_clean_fusion_data_train.append(np.delete(data_train, list_bad_feature, axis=1))
    final_clean_fusion_data_val.append(np.delete(data_val, list_bad_feature, axis=1))

final_clean_fusion_data_test = np.delete(fusion_data_test, list_bad_feature, axis=1)

In [109]:
final_clean_fusion_data_train[0].shape

(1132, 234)

In [110]:
final_clean_fusion_data_val[0].shape

(126, 234)

In [111]:
final_clean_fusion_data_test.shape

(315, 234)

In [113]:
# create dataloader for training (vector data)
#======================================================================================
list_data_fusion_train =[]
list_data_fusion_val =[]
list_data_target_train =[]
list_data_target_val =[]

for data_train, data_val, tr_targets, val_targets in zip(final_clean_fusion_data_train, final_clean_fusion_data_val,total_train_targets, total_validation_targets):
    train_loader = V_Loader(dataset = data_train, batch_size = 126)
    val_loader = V_Loader(dataset = data_val, batch_size = 126)
    
    tr_target_loader = V_Loader(dataset = tr_targets, batch_size = 126)
    val_target_loader =  V_Loader(dataset = val_targets, batch_size = 126)
    
    list_data_fusion_train.append(train_loader)
    list_data_fusion_val.append(val_loader)
    
    list_data_target_train.append(tr_target_loader)
    list_data_target_val.append(val_target_loader)

In [114]:
#criterion = torch.nn.CrossEntropyLoss()
#define the loss function 
criterion = torch.nn.BCELoss()

In [115]:
def train_1(train_v_loaderB,tr_target_v_loaderB, combined_model, optimizer):
    combined_model.train()

# only used for counting the lost function for data train
#====================================================================    
    
    for data_X2, data_target in zip(train_v_loaderB, tr_target_v_loaderB):  # Iterate in batches over the training dataset.
        out = combined_model(torch.tensor(data_X2, dtype=torch.float32))  # Perform a single forward pass.
        #print(out)
        y_t = torch.tensor(data_target, dtype=torch.float32)
        loss = criterion(out[:,0], y_t)  # Compute the loss.
        #print(k)
        #print(loss)
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return loss, combined_model, optimizer

# only used for counting the lost function for data validation
#====================================================================
def validation_1(val_v_loaderB,val_target_v_loaderB, combined_model):
    for data_X2, data_target in zip(val_v_loaderB, val_target_v_loaderB): # Iterate in batches over the training dataset.
        out = combined_model(torch.tensor(data_X2, dtype=torch.float32))  # Perform a single forward pass.
        
        y_t = torch.tensor(data_target, dtype=torch.float32)
        val_loss = criterion(out[:,0], y_t)  # Compute the loss.
        
    return val_loss

In [116]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

# performances visualization 
import matplotlib.pyplot as plt
#import seaborn as sns
import statistics
import math
from prettytable import PrettyTable
%run ./my_performances.ipynb 


def test_1(v_loaderB,v_target, combined_model):
    combined_model.eval()
    list_pred =[]
    list_targets =[]
    correct = 0
    for data_X2, data_target in zip (v_loaderB,v_target):  # Iterate in batches over the training/test dataset.
            out = combined_model(torch.tensor(data_X2, dtype=torch.float32))
            out_1 = out[:,0]
            
            list_pred.append(out_1.item())
            list_targets.append(data_target.item())
    return list_pred, list_targets

# used to count the train accuracy ,and validation accuracy when in the training mode 
def test(v_loaderB,target_v_loaderB, combined_model):
    combined_model.eval()

    correct = 0
    for data_X2, data_target in zip(v_loaderB,target_v_loaderB):  # Iterate in batches over the training/test dataset.
            out = combined_model(torch.tensor(data_X2, dtype=torch.float32))
            out_1 = out[:,0]
            for i,value in enumerate(out_1) :
                if value > 0.5 :
                    out_1[i] = 1
                else : out_1[i] = 0
            pred = out_1  # Use the class with highest probability.
            correct += int((pred == data_target).sum())  # Check against ground-truth labels.
    return correct / len(v_loaderB.dataset)  # Derive ratio of correct predictions.

In [117]:
def get_optimizer(gnn_model, learning_rate, optimizer_type, weight_decay=1e-4):
    if optimizer_type==1:
        optimizer = torch.optim.SGD(gnn_model.parameters(), lr=learning_rate, momentum=0.9)
    if optimizer_type==2:
        optimizer = torch.optim.Adam(gnn_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    if optimizer_type ==3 :
        optimizer = torch.optim.Adamax(gnn_model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4)
        
    return optimizer

In [118]:
class modelB(torch.nn.Module):
    def __init__(self, input_features, output_features,dropout_rateB1,dropout_rateB2,dropout_rateB3,  
                 dense_layer1,dense_layer2, dense_layer3):
        super(modelB, self).__init__()
        self.lin1 = nn.Linear(input_features,dense_layer1)
      
        self.lin2 = nn.Linear(int(dense_layer1), dense_layer2)
        self.lin3 = nn.Linear(int(dense_layer2), dense_layer3)
        self.lin4 = nn.Linear(int(dense_layer3), output_features)
        
        self.bn1 = nn.BatchNorm1d(int(dense_layer1))
        self.bn2 = nn.BatchNorm1d(int(dense_layer2))
        self.bn3 = nn.BatchNorm1d(int(dense_layer3))
        self.dropoutB1 = dropout_rateB1
        self.dropoutB2 = dropout_rateB2
        self.dropoutB3 = dropout_rateB3
        
    def forward(self, x):
        x = self.lin1(x)
        
        x = self.bn1(x)
       
        x = F.dropout(x, p= self.dropoutB1, training=self.training)
        x = x.relu()
  #      
        x = self.lin2(x)
        x = self.bn2(x)   
        x = F.dropout(x, p= self.dropoutB2, training=self.training)
        x = x.relu()
  #      
        x = self.lin3(x)
        x = self.bn3(x)   
        x = F.dropout(x, p= self.dropoutB3, training=self.training)

        x = x.relu()
        x = self.lin4(x)
        return torch.sigmoid(x)    

In [119]:
k=10 # number of fold 
input_featuresB    = 234 # length of feature data vector 
output_featuresB   = 1
optimizer_type = 2
dropout_rateB1 = 0.15552924759769313
dropout_rateB2 =0.207127885808487
dropout_rateB3 =0.2240506664932075

dense_layer1=60
dense_layer2=30
dense_layer3=10
learning_rate = 0.015061165279742287
weight_decay=0.00017722962529205202

list_data_vec_trainB = list_data_fusion_train
list_data_vec_valB = list_data_fusion_val

In [120]:
# APPLY THE HYPERPARAMETER 
# RUNNING EVERY FOLD 
#=============================================================

total_history_train_loss=[]
total_history_val_loss=[]
total_history_train_acc=[]
total_history_val_acc=[]
list_trained_model =[]

for i in range(k):
        
        combined_model   = modelB(input_featuresB, output_featuresB,
                                   dropout_rateB1,dropout_rateB1,dropout_rateB3, 
                                   dense_layer1,dense_layer2, dense_layer3)
        optimizer = get_optimizer(combined_model, learning_rate, optimizer_type,weight_decay)
   
        history_train_loss=[]
        history_val_loss=[]
        history_train_acc=[]
        history_val_acc=[]
        
        for epoch in range(1,40):
            
                
            train_loss,combined_model, optimizer = train_1(list_data_vec_trainB[i],list_data_target_train[i],
                                                           combined_model,optimizer)
            val_loss                             = validation_1(list_data_vec_valB[i],list_data_target_val[i],
                                                                combined_model) 
        
            train_acc                            = test(list_data_vec_trainB[i],list_data_target_train[i],combined_model)
            val_acc                              = test(list_data_vec_valB[i],list_data_target_val[i], combined_model)
        
           
            
            history_train_loss.append(train_loss.item())
            history_val_loss.append(val_loss.item())
            history_train_acc.append(train_acc)
            history_val_acc.append(val_acc)
        
            print(f'Fold: {i}')
            print(f'Epoch: {epoch:03d}, Train Loss:, {train_loss.item():.4f}')
            print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
            print(f'Epoch: {epoch:03d}, val Loss:, {val_loss.item():.4f}')
            print(f'Epoch: {epoch:03d}, val Acc: {val_acc:.4f}')
            print(" ")
        total_history_train_loss.append(history_train_loss)
        total_history_val_loss.append(history_val_loss)
        total_history_train_acc.append(history_train_acc)
        total_history_val_acc.append(history_val_acc)

        list_trained_model.append(combined_model)


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct f

Fold: 0
Epoch: 001, Train Loss:, 0.5542
Epoch: 001, Train Acc: 0.6882
Epoch: 001, val Loss:, 0.6021
Epoch: 001, val Acc: 0.7143
 
Fold: 0
Epoch: 002, Train Loss:, 0.5261
Epoch: 002, Train Acc: 0.7235
Epoch: 002, val Loss:, 0.5702
Epoch: 002, val Acc: 0.7381
 
Fold: 0
Epoch: 003, Train Loss:, 0.4794
Epoch: 003, Train Acc: 0.7438
Epoch: 003, val Loss:, 0.5693
Epoch: 003, val Acc: 0.7222
 
Fold: 0
Epoch: 004, Train Loss:, 0.4785
Epoch: 004, Train Acc: 0.7518
Epoch: 004, val Loss:, 0.5340
Epoch: 004, val Acc: 0.7619
 
Fold: 0
Epoch: 005, Train Loss:, 0.4626
Epoch: 005, Train Acc: 0.7809
Epoch: 005, val Loss:, 0.5725
Epoch: 005, val Acc: 0.7778
 
Fold: 0
Epoch: 006, Train Loss:, 0.4774
Epoch: 006, Train Acc: 0.7924
Epoch: 006, val Loss:, 0.6119
Epoch: 006, val Acc: 0.7222
 
Fold: 0
Epoch: 007, Train Loss:, 0.3320
Epoch: 007, Train Acc: 0.8242
Epoch: 007, val Loss:, 0.6149
Epoch: 007, val Acc: 0.7302
 
Fold: 0
Epoch: 008, Train Loss:, 0.4114
Epoch: 008, Train Acc: 0.8251
Epoch: 008, val Loss

Fold: 1
Epoch: 029, Train Loss:, 0.1802
Epoch: 029, Train Acc: 0.9346
Epoch: 029, val Loss:, 2.1937
Epoch: 029, val Acc: 0.6508
 
Fold: 1
Epoch: 030, Train Loss:, 0.2342
Epoch: 030, Train Acc: 0.9532
Epoch: 030, val Loss:, 1.9622
Epoch: 030, val Acc: 0.6825
 
Fold: 1
Epoch: 031, Train Loss:, 0.2227
Epoch: 031, Train Acc: 0.9470
Epoch: 031, val Loss:, 2.0982
Epoch: 031, val Acc: 0.6667
 
Fold: 1
Epoch: 032, Train Loss:, 0.1462
Epoch: 032, Train Acc: 0.9505
Epoch: 032, val Loss:, 1.2638
Epoch: 032, val Acc: 0.6825
 
Fold: 1
Epoch: 033, Train Loss:, 0.2385
Epoch: 033, Train Acc: 0.9541
Epoch: 033, val Loss:, 1.4476
Epoch: 033, val Acc: 0.6825
 
Fold: 1
Epoch: 034, Train Loss:, 0.1736
Epoch: 034, Train Acc: 0.9629
Epoch: 034, val Loss:, 1.3818
Epoch: 034, val Acc: 0.7063
 
Fold: 1
Epoch: 035, Train Loss:, 0.1100
Epoch: 035, Train Acc: 0.9647
Epoch: 035, val Loss:, 2.0332
Epoch: 035, val Acc: 0.6984
 
Fold: 1
Epoch: 036, Train Loss:, 0.1462
Epoch: 036, Train Acc: 0.9602
Epoch: 036, val Loss

Fold: 3
Epoch: 018, Train Loss:, 0.3647
Epoch: 018, Train Acc: 0.9337
Epoch: 018, val Loss:, 1.2304
Epoch: 018, val Acc: 0.6587
 
Fold: 3
Epoch: 019, Train Loss:, 0.2427
Epoch: 019, Train Acc: 0.9293
Epoch: 019, val Loss:, 1.4530
Epoch: 019, val Acc: 0.6270
 
Fold: 3
Epoch: 020, Train Loss:, 0.1894
Epoch: 020, Train Acc: 0.9417
Epoch: 020, val Loss:, 1.1591
Epoch: 020, val Acc: 0.6429
 
Fold: 3
Epoch: 021, Train Loss:, 0.1952
Epoch: 021, Train Acc: 0.9329
Epoch: 021, val Loss:, 1.3345
Epoch: 021, val Acc: 0.6111
 
Fold: 3
Epoch: 022, Train Loss:, 0.2264
Epoch: 022, Train Acc: 0.9355
Epoch: 022, val Loss:, 1.3670
Epoch: 022, val Acc: 0.6508
 
Fold: 3
Epoch: 023, Train Loss:, 0.2155
Epoch: 023, Train Acc: 0.9417
Epoch: 023, val Loss:, 1.4718
Epoch: 023, val Acc: 0.6587
 
Fold: 3
Epoch: 024, Train Loss:, 0.2518
Epoch: 024, Train Acc: 0.9417
Epoch: 024, val Loss:, 1.5581
Epoch: 024, val Acc: 0.6746
 
Fold: 3
Epoch: 025, Train Loss:, 0.1973
Epoch: 025, Train Acc: 0.9417
Epoch: 025, val Loss

Fold: 5
Epoch: 008, Train Loss:, 0.4060
Epoch: 008, Train Acc: 0.8595
Epoch: 008, val Loss:, 0.7602
Epoch: 008, val Acc: 0.7143
 
Fold: 5
Epoch: 009, Train Loss:, 0.3082
Epoch: 009, Train Acc: 0.8542
Epoch: 009, val Loss:, 0.7129
Epoch: 009, val Acc: 0.6905
 
Fold: 5
Epoch: 010, Train Loss:, 0.2878
Epoch: 010, Train Acc: 0.8852
Epoch: 010, val Loss:, 0.7935
Epoch: 010, val Acc: 0.6905
 
Fold: 5
Epoch: 011, Train Loss:, 0.3658
Epoch: 011, Train Acc: 0.8958
Epoch: 011, val Loss:, 0.9642
Epoch: 011, val Acc: 0.6905
 
Fold: 5
Epoch: 012, Train Loss:, 0.2510
Epoch: 012, Train Acc: 0.8905
Epoch: 012, val Loss:, 0.8259
Epoch: 012, val Acc: 0.6508
 
Fold: 5
Epoch: 013, Train Loss:, 0.2967
Epoch: 013, Train Acc: 0.9028
Epoch: 013, val Loss:, 0.8984
Epoch: 013, val Acc: 0.7222
 
Fold: 5
Epoch: 014, Train Loss:, 0.2547
Epoch: 014, Train Acc: 0.9090
Epoch: 014, val Loss:, 0.8911
Epoch: 014, val Acc: 0.6905
 
Fold: 5
Epoch: 015, Train Loss:, 0.1914
Epoch: 015, Train Acc: 0.8975
Epoch: 015, val Loss

Fold: 6
Epoch: 036, Train Loss:, 0.1424
Epoch: 036, Train Acc: 0.9576
Epoch: 036, val Loss:, 1.8718
Epoch: 036, val Acc: 0.5794
 
Fold: 6
Epoch: 037, Train Loss:, 0.0957
Epoch: 037, Train Acc: 0.9585
Epoch: 037, val Loss:, 1.8656
Epoch: 037, val Acc: 0.5714
 
Fold: 6
Epoch: 038, Train Loss:, 0.2160
Epoch: 038, Train Acc: 0.9488
Epoch: 038, val Loss:, 1.7668
Epoch: 038, val Acc: 0.5952
 
Fold: 6
Epoch: 039, Train Loss:, 0.1676
Epoch: 039, Train Acc: 0.9532
Epoch: 039, val Loss:, 1.7154
Epoch: 039, val Acc: 0.5714
 
Fold: 7
Epoch: 001, Train Loss:, 0.5971
Epoch: 001, Train Acc: 0.7014
Epoch: 001, val Loss:, 0.6176
Epoch: 001, val Acc: 0.6667
 
Fold: 7
Epoch: 002, Train Loss:, 0.5259
Epoch: 002, Train Acc: 0.6996
Epoch: 002, val Loss:, 0.6121
Epoch: 002, val Acc: 0.6825
 
Fold: 7
Epoch: 003, Train Loss:, 0.4892
Epoch: 003, Train Acc: 0.7306
Epoch: 003, val Loss:, 0.5942
Epoch: 003, val Acc: 0.7063
 
Fold: 7
Epoch: 004, Train Loss:, 0.4570
Epoch: 004, Train Acc: 0.7650
Epoch: 004, val Loss

Fold: 8
Epoch: 023, Train Loss:, 0.1930
Epoch: 023, Train Acc: 0.9532
Epoch: 023, val Loss:, 2.1403
Epoch: 023, val Acc: 0.6000
 
Fold: 8
Epoch: 024, Train Loss:, 0.1926
Epoch: 024, Train Acc: 0.9356
Epoch: 024, val Loss:, 1.2513
Epoch: 024, val Acc: 0.6240
 
Fold: 8
Epoch: 025, Train Loss:, 0.1361
Epoch: 025, Train Acc: 0.9453
Epoch: 025, val Loss:, 1.1009
Epoch: 025, val Acc: 0.6720
 
Fold: 8
Epoch: 026, Train Loss:, 0.1354
Epoch: 026, Train Acc: 0.9585
Epoch: 026, val Loss:, 1.1785
Epoch: 026, val Acc: 0.6400
 
Fold: 8
Epoch: 027, Train Loss:, 0.1710
Epoch: 027, Train Acc: 0.9594
Epoch: 027, val Loss:, 1.5598
Epoch: 027, val Acc: 0.6480
 
Fold: 8
Epoch: 028, Train Loss:, 0.1603
Epoch: 028, Train Acc: 0.9409
Epoch: 028, val Loss:, 1.3357
Epoch: 028, val Acc: 0.6240
 
Fold: 8
Epoch: 029, Train Loss:, 0.1725
Epoch: 029, Train Acc: 0.9479
Epoch: 029, val Loss:, 1.3652
Epoch: 029, val Acc: 0.6800
 
Fold: 8
Epoch: 030, Train Loss:, 0.2270
Epoch: 030, Train Acc: 0.9532
Epoch: 030, val Loss

In [122]:
# RUNNING THE 10 PRETRAINED MODEL USING INDEPENDENT DATA
#======================================================================
nCV= 10 # ten crossfold validation 
list_fold_pred =[]
list_fold_targets =[]


v_test_loaderB = V_Loader(dataset = final_clean_fusion_data_test, batch_size = 1)
v_test_target = V_Loader(dataset = total_test_targets, batch_size = 1)

for combined_model in list_trained_model:  
    list_pred, list_targets = test_1(v_test_loaderB,v_test_target,combined_model)
    list_fold_pred.append(list_pred)
    list_fold_targets.append(list_targets)
    


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [123]:
# GET THE PERFORMANCES FROM THE TEST
#========================================================================
total_performances = performances(list_fold_pred, list_fold_targets, nCV)
list_bal_acc = []
for sen, spec in zip (total_performances[1] , total_performances[2]):
    bal_acc = (sen + spec)/2
    list_bal_acc.append(bal_acc)
                
statistics.mean(list_bal_acc)

0.680955

In [124]:
import statistics
from prettytable import PrettyTable
perf = total_performances
model_title = 'Test Perf FCNN'
data_type ='Fusion'
Create_Tables(perf, model_title, data_type)

+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
|   Model Name   | Data Type | m_ACC |  m_SN |  m_SP | m_MCC | m_AUC | m_Kappa | m_AUROC | m_Bal_ACC |  m_F1 |
+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
| Test Perf FCNN |   Fusion  | 0.723 | 0.794 | 0.568 |  0.36 | 0.742 |  0.359  |  0.742  |   0.681   | 0.798 |
+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
|   Model Name   | Data Type | e_ACC |  e_SN |   SP  | e_MCC | e_AUC | e_Kappa | e_AUROC | e_Bal_ACC |  e_F1 |
+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
| Test Perf FCNN |   Fusion  | 0.015 | 0.025 | 0.038 | 0.032 | 0.014 |  0.031  |  0.014  |   0.016   | 0.798 |
+